# Get reconstruction provenance

- Created with PtyRAD 0.1.0b13
- Requires PtyRAD >= 0.1.0b13
- Documentation: https://ptyrad.readthedocs.io/en/latest/
- PtyRAD paper: https://doi.org/10.1093/mam/ozaf070
- PtyRAD arXiv: https://arxiv.org/abs/2505.07814
- Zenodo record: https://doi.org/10.5281/zenodo.15273176
- Box folder: https://cornell.box.com/s/n5balzf88jixescp9l15ojx7di4xn1uo
- Youtube channel: https://www.youtube.com/@ptyrad_official

**Before running this notebook, you must first follow the instruction in `README.md` to:**
1. Create the Python environment with all dependant Python packages like PyTorch
2. Activate that python environment
3. Install `ptyrad` package into your activated Python environement (only need to install once)

> Note: This notebook is designed for retrieving and exporting the full reconstruction history (i.e., provenance) for sequential reconstructions

Author: Chia-Hao Lee, cl2696@cornell.edu

## Why we want the full provenance record?

It's very common to run sequential reconstructions to refine your ptychography results.

For example, we might do the following:
1. Run a reduced FOV using 1 nm slice
2. Load the previous probe from run #1, run a new reconstruction for full FOV
3. Take the previous probe, object, and positions from run #2, and run a new reconstruction with 0.5 nm slice
4. Take the previous probe, object, and positions from run #3, and further reduce the depth regularization strength

For full reproducibility, one needs to keep track of all the output HDF5s and param files. This process is quite demanding and error prone, often leads to incorrect or incomplete record if not done carefully. Therefore, a light-weight automatic provenance tracking feature is highly desirable.

## How PtyRAD resolve it?
PtyRAD collects the metadata and params for each run and saves it as the provenance record in the HDF5 output. For sequantial runs, it will collect the provenance record during initialization (specifically, while loading previous runs), and combine with the current record while saving the HDF5 output. This way, the full history of metadata and participating params are inherited, making it much easier to reproduce results from sequential reconstructions.

## 00. Setup working directory and imports

In [ ]:
import os

# Set this to your desired working directory so you can easily access the data, model, param files
work_dir = "H:/workspace/ptyrad/"

os.chdir(work_dir)
print("Current working dir: ", os.getcwd())
# Note that the output/ directory will be automatically generated under your working directory

In [ ]:
from ptyrad.io.load import load_ptyrad
from ptyrad.io.provenance import load_provenance_from_h5, export_hdf5_provenance_to_json

## 01. Load PtyRAD HDF5 output file

In [ ]:
hdf5_path   = "output/tBL_WSe2/20260209_full_N16384_dp128_flipT100_random32_p6_1obj_6slice_dz2_orblur0.4_ozblur1.0_oposc_sng1.0_spr0.1/model_iter0003.hdf5"
ckpt         = load_ptyrad(hdf5_path)

# The reconstructed arrays (probe, object, positions) and metadata are stored as HDF5 groups/datasets and parsed as Python dict.
print("ckpt: ")
for k in ckpt.keys():
    print('  ', k)

## 02. Load the provenance record as Python dict from HDF5 attr

In [ ]:
provenance = load_provenance_from_h5(hdf5_path)
provenance

## 03. Export the provenance record as JSON from HDF5 attr

In [ ]:
export_hdf5_provenance_to_json(hdf5_path, output_json_path=None); # If output_json_path=None, the JSON would be output to the same folder with hdf5_path)

## 04. Understanding Your Provenance Record
Now that you have exported the provenance history, you might wonder how to read it. The structure is designed to answer one question: "Where did this component come from, and what happened to it along the way?"

To understand it, think of your reconstruction as a **Family Tree**.

### a. The Bloodlines (Components)

Your reconstruction is the "child" of multiple parents. The Probe, Object, Positions, and Tilts each have their own separate family history.

The Probe might be an "heir" to a long line of previous refinements (Run A → Run B → Current Run).

The Object might be a "newcomer," freshly imported from a simulation or an external file.

In the JSON file, these are stored as separate lists:

```text
{
  "probe": [ ...history... ],
  "object": [ ...history... ],
  "pos":    [ ...history... ]
}
```

### b. The Timeline (The List)
Each list represents a chronological timeline for that specific component.

- **The Genesis (Start)**: The first entry in the list tells you how the component was born. Was it simulated? Was it loaded from a legacy file? Was it a raw NumPy array?

- **The Inheritance (Middle)**: As you load previous results (e.g., using `probe_source='PtyRAD'`), the new run inherits the entire history list from the parent file. The history flows from the old file into the new one automatically.

- **The Current Run (End)**: The final entry in the list is always THIS run. It stamps the component with the parameters used in the current reconstruction. - 

**Therefore, the involved files, parameters, and the full history are all encapsulated in "1 single HDF5 file".**


### c. Reading a Trace
Here is how to read a typical trace for a Probe that has gone through 3 sequential runs:

> **Entry 1 (Grandparent):** `Simulated`
>
>   - Meaning: The probe started life as a simulation with 300kV and 20 mrad convergence angle.
>
> **Entry 2 (Parent):** `Run_Step1.h5`
>
>   - Meaning: The simulated probe was used in "Step 1" to refine a coarse image. The specific parameters (learning rate, iterations) are stored here.
>
> **Entry 3 (Current Run):** `Run_Step2.h5`
>
>   - Meaning: We loaded the result from Step 1 and refined it further in "Step 2". **This file contains the result of this step.**

## 05. Example JSON output

```JSON
{
  "probe": [
    {
      "uid": "62c2e472",
      "timestamp": "2026-01-28T01:36:35.908309",
      "run_name": "<Simulation>",
      "action": "Simulated",
      "metadata": {
        "sim_params": null
      }
    },
    {
      "uid": "9f15b82d",
      "timestamp": "2026-01-28T01:36:40.968223",
      "run_name": "h:\\workspace\\ptyrad\\demo\\output\\tBL_WSe2\\20260128_full_N1024_dp128_flipT100_random32_p6_1obj_6slice_dz2_plr1e-4_oalr5e-4_oplr5e-4_slr5e-4_orblur0.4_ozblur1.0_mamp0.03_4.0_oathr0.96_oposc_sng1.0_spr0.1_new\\model_iter0001.hdf5",
      "note": "PtyRAD Run",
      "params": {"NEGLECTED FOR CLEANER NOTEBOOK DISPLAY"},
    },
    {
      "uid": "3cbfa081",
      "timestamp": "2026-01-28T01:45:09.575841",
      "run_name": "h:\\workspace\\ptyrad\\demo\\output\\tBL_WSe2\\20260128_full_N1024_dp128_flipT100_random32_p6_1obj_6slice_dz2_plr1e-4_oalr5e-4_oplr5e-4_slr5e-4_orblur0.4_ozblur1.0_mamp0.03_4.0_oathr0.96_oposc_sng1.0_spr0.1_new_r2\\model_iter0005.hdf5",
      "note": "PtyRAD Run",
      "params": {"NEGLECTED FOR CLEANER NOTEBOOK DISPLAY"},
    },
    {
      "uid": "bcaa4222",
      "timestamp": "2026-01-28T11:25:15.814207",
      "run_name": "h:\\workspace\\ptyrad\\demo\\output\\tBL_WSe2\\20260128_full_N16384_dp128_flipT100_random32_p6_1obj_6slice_dz2_plr1e-4_oalr5e-4_oplr5e-4_slr5e-4_orblur0.4_ozblur1.0_mamp0.03_4.0_oathr0.96_oposc_sng1.0_spr0.1_test1\\model_iter0003.hdf5",
      "note": "PtyRAD Run",
      "params": {"NEGLECTED FOR CLEANER NOTEBOOK DISPLAY"},
    }
```